In [14]:
# Importing libraries
import os 
import scipy.io
import numpy as np
import pandas as pd
from sklearn import model_selection, svm, naive_bayes
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pickle
import serial
import time
import pyautogui
from sklearn.preprocessing import MinMaxScaler 
import warnings

directory = '../Data/murillo'
filenames = [ 'punch', 'rest', 'thumbs_up', 'victory']
warnings.filterwarnings('ignore')

np.random.seed(0)

In [15]:
# Feature generation
threshold_multiplication = 1950*2100
threshold_minus = 2100-1900
def sgn(x1, x2):
    if(x1*x2 >= threshold_multiplication and abs(x1-x2) >= threshold_minus):
        return 1
    else:
        return 0

dataframe = []

def feature_extraction(xiao_data):
    rms = np.sqrt(np.mean(xiao_data**2))
    mav = np.sum(abs(xiao_data))/len(xiao_data)
    wl = np.sum([xiao_data[n] - xiao_data[n-1] for n in range(1, len(xiao_data)-1)])
    intensity = np.log(rms/len(xiao_data))
    zc = np.sum([sgn(xiao_data[n], xiao_data[n+1]) for n in range(1, len(xiao_data)-1)])
    var = np.sum(xiao_data**2)/(len(xiao_data)-1)
    ssc = np.sum([xiao_data[n] - xiao_data[n-1] if (xiao_data[n] - xiao_data[n-1])>=0 else 0  for n in range(1, len(xiao_data) - 1)])  
    return [rms, mav, wl, intensity, zc, var, ssc]

for file in filenames:
    with open(os.path.join(directory, f'{file}_continuous.txt'), "r") as f:
        lines = f.readlines()
    # extract the values after "->" and convert them to integers
    values = [int(float(line.split("->")[1].strip())) for line in lines]
    for i in range(0, len(values)-200, 150):
        x = values[0+i:200 + i]
        x = np.array(x, dtype=np.float64)
        x = x -  np.mean(x)
        feature_data = feature_extraction(x)
        feature_data.append(file)
        dataframe.append(feature_data)
            
df = pd.DataFrame(dataframe, columns=["RMS", "MAV", "WL", "INTENSITY", "ZC", "VAR", "SSC", "LABEL"])
df.head()

,RMS,MAV,WL,INTENSITY,ZC,VAR,SSC,LABEL
0,72.695053,55.97905,54.0,-1.012044,0,5311.126407,4101.0,punch
1,244.363455,192.60775,-585.0,0.200339,0,60013.565804,14998.0,punch
2,280.498644,221.99850,-132.0,0.338252,0,79074.863417,22122.0,punch
3,258.911153,199.18745,-812.0,0.258168,0,67371.844196,17885.0,punch
4,228.860984,184.00265,-91.0,0.134797,0,52640.552739,16048.0,punch


In [3]:
# Training-Test Split
X = df.drop('LABEL', axis=1)
y = df['LABEL']

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

Encoder = LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

In [4]:
# Training SVM
SVM = svm.SVC(kernel='linear',gamma='auto')
SVM.fit(X_train, y_train) 
predictions_SVM = SVM.predict(X_test)
print("SVM Accuracy Score -> {accuracy:.1f}%".format(accuracy = accuracy_score(predictions_SVM, y_test)*100))

SVM Accuracy Score -> 88.5%


In [16]:
# Training Naive Bayes
Naive = naive_bayes.MultinomialNB()
scaler = MinMaxScaler()
Naive.fit(scaler.fit_transform(X_train.values),y_train)# predict the labels on validation dataset
predictions_NB = Naive.predict(scaler.transform(X_test))# Use accuracy_score function to get the accuracy
print("NB Accuracy Score -> {accuracy:.1f}%".format(accuracy = accuracy_score(predictions_NB, y_test)*100))

NB Accuracy Score -> 54.9%


In [11]:
# Training ANN
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()
clf.fit(X_train, y_train)

predictions_NN = clf.predict(X_test)
print("Neural network Accuracy Score -> {accuracy:.1f}%".format(accuracy = accuracy_score(predictions_NN, y_test)*100))

Neural network Accuracy Score -> 68.1%


In [13]:
# Saving the best model
model_to_be_save = SVM
f = open('classifier.pickle', 'wb')
pickle.dump(model_to_be_save, f)
f.close()